## Model Training: Convolutional Neural Network (CNN)

The purpose of this notebook is to train several Convolutional Neural Network (CNN) models for earthquake detection and property prediction. Rather than using the raw seismic signals as training data, we use images of the earthquake waveforms and spectrograms (visual representations of the spectrum of frequencies in the signal) that were created using the `Data_Preprocessing` module and the `data_preprocessing.ipynb` notebook in this repository. Using these images, we train and evaluate the following models:

1. Classification CNN model to predict whether a signal is 'earthquake' or 'noise'
2. Regression CNN model to predict earthquake magnitude
3. Regression CNN model to predict earthquake p-wave arrival time
4. Regression CNN model to predict earthquake s-wave arrival time

Model architectures used in this notebook are located in the `architectures` module of this repo. Training utilities including train/val/test split data preparation, evaluation functions, and plotting functions are located in the `training_utils` module of this repo.

### Environment setup

In [1]:
import gc
import sys
import time

sys.path.append('../')
sys.path.append('../../../')

import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from PIL import Image
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mean_squared_error, roc_curve, auc
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler

import earthquake_detection.architectures as architectures
import earthquake_detection.training_utils as training_utils

plt.ioff() # Turn off matplotlib interactive mode. Prevents  memory leakage.

2025-01-29 08:50:11.863625: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Load datasets

These datasets were pre-processed from their raw STEAD format using the `DataPreprocessing` module and `data_preprocessing` notebook contained within this repo.

In [2]:
# Load extracted raw signals
raw_signals = np.load('../../../data/STEAD/extracted_raw_signals_subsample_100000.npy')

# Load created waveform images
waveform_imgs = np.load('../../../data/STEAD/created_waveform_images_subsample_100000.npy')

# Load created spectrogram images
spectrogram_imgs = np.load('../../../data/STEAD/created_spectrogram_images_subsample_100000.npy')

# Load metadata
metadata = pd.read_feather('../../../data/STEAD/extracted_metadata_subsample_100000.feather')
metadata = metadata.reset_index()

#### Inspect image datasets and metadata to confirm they look as expected

In [3]:
spectrogram_imgs.shape

(100000, 200, 300, 3)

In [4]:
waveform_imgs.shape

(100000, 100, 300, 3)

In [5]:
metadata

,trace_name,network_code,receiver_code,receiver_type,receiver_latitude,receiver_longitude,receiver_elevation_m,p_arrival_sample,p_status,p_weight,...,source_mechanism_strike_dip_rake,source_distance_deg,source_distance_km,back_azimuth_deg,snr_db,coda_end_sample,trace_start_time,trace_category,chunk,weight_for_subsample
0,B084.PB_20111212104350_EV,PB,B084,EH,33.611570,-116.456370,1271.0,800.0,manual,0.55,...,None,0.5746,63.90,178.5,[15.39999962 15.5 17.10000038],[[3199.]],43:51.3,earthquake_local,1,9.706561e-07
1,B086.PB_20080618145426_EV,PB,B086,EH,33.557500,-116.531000,1392.0,800.0,manual,0.59,...,None,0.5071,56.40,330.6,[45. 46.20000076 46.29999924],[[2230.]],54:27.0,earthquake_local,1,9.706561e-07
2,B023.PB_20130513182210_EV,PB,B023,EH,46.111200,-123.078700,177.4,500.0,manual,0.63,...,None,0.9683,107.61,163.1,[12.60000038 12.10000038 11.19999981],[[3199.]],22:11.5,earthquake_local,1,9.706561e-07
3,B011.PB_20120724202107_EV,PB,B011,EH,48.649543,-123.448192,22.0,800.0,manual,0.76,...,None,0.8950,99.44,82.7,[20.39999962 26.39999962 25. ],[[3199.]],21:08.5,earthquake_local,1,9.706561e-07
4,B082.PB_20150914105733_EV,PB,B082,HH,33.598182,-116.596005,1374.8,799.0,autopicker,0.92,...,None,0.1618,17.99,123.9,[28.79999924 30.29999924 28.70000076],[[1516.]],57:35.0,earthquake_local,1,9.706561e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,NGVB.NC_200410262236_NO,NC,NGVB,HN,38.280430,-122.215820,234.0,NaN,None,NaN,...,None,NaN,NaN,NaN,None,None,2004-10-26 22:36:00,noise,6,4.247619e-06
99996,DSP.NN_201704270525_NO,NN,DSP,HH,37.369400,-117.974100,2400.0,NaN,None,NaN,...,None,NaN,NaN,NaN,None,None,2017-04-27 05:25:00,noise,6,4.247619e-06
99997,CHR.NC_201005240430_NO,NC,CHR,HN,37.372880,-121.768870,489.0,NaN,None,NaN,...,None,NaN,NaN,NaN,None,None,2010-05-24 04:30:00,noise,6,4.247619e-06
99998,ASTN.ET_20180116092824_NO,ET,ASTN,EH,36.327000,-83.476000,753.0,NaN,None,NaN,...,None,NaN,NaN,NaN,None,None,2018-01-16 09:28:24,noise,6,4.247619e-06


In [6]:
metadata.columns

Index(['trace_name', 'network_code', 'receiver_code', 'receiver_type',
       'receiver_latitude', 'receiver_longitude', 'receiver_elevation_m',
       'p_arrival_sample', 'p_status', 'p_weight', 'p_travel_sec',
       's_arrival_sample', 's_status', 's_weight', 'source_id',
       'source_origin_time', 'source_origin_uncertainty_sec',
       'source_latitude', 'source_longitude', 'source_error_sec',
       'source_gap_deg', 'source_horizontal_uncertainty_km', 'source_depth_km',
       'source_depth_uncertainty_km', 'source_magnitude',
       'source_magnitude_type', 'source_magnitude_author',
       'source_mechanism_strike_dip_rake', 'source_distance_deg',
       'source_distance_km', 'back_azimuth_deg', 'snr_db', 'coda_end_sample',
       'trace_start_time', 'trace_category', 'chunk', 'weight_for_subsample'],
      dtype='object')

### Create subsets of earthquake-only data (no noise samples)
This is done in preparation for later training of regression models to predict earthquake parameters.

In [7]:
earthquakes_indices = np.array(list(metadata[metadata['trace_category']=='earthquake_local'].index))
raw_signals_earthquakes_only = raw_signals[earthquakes_indices]
waveform_imgs_earthquakes_only = waveform_imgs[earthquakes_indices]
spectrogram_imgs_earthquakes_only = spectrogram_imgs[earthquakes_indices]
metadata_earthquakes_only = metadata[metadata['trace_category']=='earthquake_local']

### Create labels for the classification model

In [8]:
metadata['label'] = [1 if label=='earthquake_local' else 0 for label in metadata['trace_category']]
classifier_labels = metadata['label'].values

In [9]:
classifier_labels

array([1, 1, 1, ..., 0, 0, 0])

### Parameter setup

In [10]:
spectrogram_kwargs = {'image_size' : (150,100)}
waveform_kwargs = {'image_size' : (300,100)}

## Model training & evaluation

### 1. Classification CNN model training & evaluation to predict whether signals are 'earthquake' or 'noise'

In [ ]:
# Set up model callbacks
epochs = 50
callbacks_c = architectures.callbacks_setup(model_tag='cnn_classification', epochs=epochs)

# Fit model
train_dataset_c, val_dataset_c, test_dataset_c = training_utils.prepare_datasets(imgs=spectrogram_imgs, labels=classifier_labels, preproc_func=architectures.image_preprocessing, preproc_func_kwargs=spectrogram_kwargs, batch_size=32)
classifier_model = architectures.build_compile_classification_cnn(learning_rate=1e-5, loss='binary_crossentropy', metrics=['accuracy'])
classifier_history = classifier_model.fit(train_dataset_c, epochs=epochs, callbacks=callbacks_c, validation_data=val_dataset_c)

# Evaluate model and plot results
training_utils.evaluate_classification_model(classifier_model, train_dataset_c, test_dataset_c)
training_utils.plot_training_history(classifier_history)
classifier_model.summary()

del train_dataset_c
del val_dataset_c
del test_dataset_c
del spectrogram_imgs
gc.collect()

Epoch 1/50
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 411s 202ms/step - accuracy: 0.9413 - loss: 0.1773 - val_accuracy: 0.9794 - val_loss: 0.0600 - learning_rate: 1.0000e-05
Epoch 2/50
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 407s 203ms/step - accuracy: 0.9779 - loss: 0.0657 - val_accuracy: 0.9834 - val_loss: 0.0516 - learning_rate: 1.0000e-05
Epoch 3/50
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 407s 203ms/step - accuracy: 0.9819 - loss: 0.0535 - val_accuracy: 0.9869 - val_loss: 0.0421 - learning_rate: 1.0000e-05
Epoch 4/50
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 407s 203ms/step - accuracy: 0.9851 - loss: 0.0467 - val_accuracy: 0.9822 - val_loss: 0.0484 - learning_rate: 1.0000e-05
Epoch 5/50
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 409s 204ms/step - accuracy: 0.9866 - loss: 0.0412 - val_accuracy: 0.9879 - val_loss: 0.0366 - learning_rate: 1.0000e-05
Epoch 6/50
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 409s 204ms/step - accuracy: 0.9885 - loss: 0.0361 - val_accuracy: 0.9833 - val_loss: 0.0489 - learning_rate: 1.0000e-05
Epoch 7/50
2000/2000 ━━━━━━━

### 2. Regression CNN model training & evaluation to predict earthquake magnitude

In [ ]:
epochs = 20
callbacks_mag = architectures.callbacks_setup(model_tag='cnn_regression_magnitude', epochs=epochs)

labels = np.array(metadata_earthquakes_only['source_magnitude'])
train_dataset_mag, val_dataset_mag, test_dataset_mag = training_utils.prepare_datasets(imgs=spectrogram_imgs_earthquakes_only, labels=labels, preproc_func=architectures.image_preprocessing, preproc_func_kwargs=spectrogram_kwargs, use_scaler=False, batch_size=64)
model_magnitude = architectures.build_compile_regression_cnn(learning_rate=1e-4, loss='mse', metrics=['mae'])
model_magnitude_history = model_magnitude.fit(train_dataset_mag, epochs=epochs, callbacks=callbacks_mag, validation_data=val_dataset_mag)

training_utils.evaluate_regression_model(model_magnitude, train_dataset_mag, test_dataset_mag, 'Source Magnitude', scaler=None)
training_utils.plot_training_history(model_magnitude_history)
model_magnitude.summary()

del train_dataset_mag
del val_dataset_mag
del test_dataset_mag
del spectrogram_imgs_earthquakes_only
gc.collect()

### 3. Regression CNN model training & evaluation to predict earthquake P-wave arrival time

In [ ]:
# Set up model callbacks
epochs = 20
callbacks_pwave = architectures.callbacks_setup(model_tag='cnn_regression_pwave', epochs=epochs)

# Fit model
labels = np.array(metadata_earthquakes_only['p_arrival_sample'])
train_dataset_pwave, val_dataset_pwave, test_dataset_pwave = training_utils.prepare_datasets(imgs=waveform_imgs_earthquakes_only, labels=labels, preproc_func=architectures.image_preprocessing, preproc_func_kwargs=waveform_kwargs, use_scaler=False, batch_size=32)
model_pwave = architectures.build_compile_regression_cnn(learning_rate=1e-4, loss='mse', metrics=['mae'])
model_pwave_history = model_pwave.fit(train_dataset_pwave, epochs=epochs, callbacks=callbacks_pwave, validation_data=val_dataset_pwave)

# Evaluate model
training_utils.evaluate_regression_model(model_pwave, train_dataset_pwave, test_dataset_pwave, 'P-Wave Arrival', scaler=None)
training_utils.plot_training_history(model_pwave_history)
model_pwave.summary()

del train_dataset_pwave
del val_dataset_pwave
del test_dataset_pwave
gc.collect()

### 4. Regression CNN model training & evaluation to predict earthquake S-wave arrival time

In [ ]:
# Set up model callbacks
epochs = 20
callbacks_swave = architectures.callbacks_setup(model_tag='cnn_regression_swave', epochs=epochs)

# Fit model
labels = np.array(metadata_earthquakes_only['s_arrival_sample'])
train_dataset_swave, val_dataset_swave, test_dataset_swave = training_utils.prepare_datasets(imgs=waveform_imgs_earthquakes_only, labels=labels, preproc_func=architectures.image_preprocessing, preproc_func_kwargs=waveform_kwargs, use_scaler=False, batch_size=32)
model_swave = architectures.build_compile_regression_cnn(learning_rate=1e-4, loss='mse', metrics=['mae'])
model_swave_history = model_swave.fit(train_dataset_swave, epochs=epochs, callbacks=callbacks_swave, validation_data=val_dataset_swave)

# Evaluate model
training_utils.evaluate_regression_model(model_swave, train_dataset_swave, test_dataset_swave, 'S-Wave Arrival', scaler=None)
training_utils.plot_training_history(model_swave_history)
model_swave.summary()

del train_dataset_swave
del val_dataset_swave
del test_dataset_swave
gc.collect()